In [71]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
from dotenv import load_dotenv
              
import requests
              
import json
from pandas.io.json import json_normalize

import numpy as np



In [2]:
load_dotenv()
client_id = os.getenv('foursquare_id')
client_secret = os.getenv('foursquare_secret')
version = '20200508'


In [3]:
dallas_df = pd.read_csv('./Data/dfw_demographics.csv')

In [4]:
dallas_df.head()

,Zip_Code,Neighborhood,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,75001,Addison,32.96,-96.84,12414,57633.0,258200.0,0.092103,0.366306,0.038184,0.332532,0.144217,0.023251,0.003407
1,75002,West Allen,33.08,-96.61,63140,91632.0,186100.0,0.052425,0.416282,0.085043,0.310966,0.109292,0.015340,0.010652
2,75006,Central Carrolton,32.95,-96.89,46364,56985.0,146800.0,0.252946,0.383821,0.066069,0.209083,0.075295,0.010197,0.002590
3,75007,North Carrollton,33.00,-96.90,51624,78934.0,169400.0,0.101158,0.426112,0.081565,0.288381,0.075876,0.014833,0.012075
4,75009,Celina,33.30,-96.80,8785,87581.0,190500.0,0.073749,0.525198,0.106760,0.215979,0.069008,0.006146,0.003161


In [5]:
dallas_df.tail()

,Zip_Code,Neighborhood,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
265,76487,Poolville,33.000,-97.90,2507,54872.0,132800.0,0.151007,0.605369,0.070470,0.141611,0.031544,0.000000,0.0
266,76490,Whitt,32.957,-98.02,91,107344.0,32500.0,0.172840,0.753086,0.074074,0.000000,0.000000,0.000000,0.0
267,76623,Avalon,32.300,-96.80,242,41250.0,80700.0,0.410526,0.536842,0.000000,0.052632,0.000000,0.000000,0.0
268,76651,Italy,32.200,-96.90,3300,50457.0,90300.0,0.224719,0.588764,0.048989,0.106067,0.029663,0.001798,0.0
269,76670,Milford,32.100,-97.00,1377,49667.0,86700.0,0.192175,0.660529,0.036824,0.100115,0.005754,0.004603,0.0


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius= 16100, limit = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)  

In [9]:
dallas_venues = get_nearby_venues(names = dallas_df['Neighborhood'],
                                   latitudes = dallas_df['lat'],
                                   longitudes = dallas_df['lng']
                                  )

In [10]:
dallas_venues.shape

(23407, 7)

In [11]:
dallas_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addison,32.96,-96.84,Taste of Chicago,32.950202,-96.839735,Pizza Place
1,Addison,32.96,-96.84,Addison Athletic Club,32.949904,-96.849136,Gym
2,Addison,32.96,-96.84,Addison Circle Park,32.960917,-96.826488,Park
3,Addison,32.96,-96.84,Big D Party Rentals,32.977245,-96.838521,Shop & Service
4,Addison,32.96,-96.84,Best Thai Signature,32.954790,-96.841320,Thai Restaurant


In [12]:
dallas_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
23402,Milford,32.1,-97.0,Shell,32.195247,-96.897644,Gas Station
23403,Milford,32.1,-97.0,Rest Area,32.153670,-97.126479,Rest Area
23404,Milford,32.1,-97.0,Taco Bell,32.196429,-96.900353,Fast Food Restaurant
23405,Milford,32.1,-97.0,Grandy's,32.196979,-96.899930,Diner
23406,Milford,32.1,-97.0,Monolithic Domes Bruco,32.197193,-96.898630,Scenic Lookout


In [13]:
dallas_venues.to_csv('./Data/dallas_venues.csv', index = False)

In [14]:
houston_df = pd.read_csv('./Data/houston_demographics.csv')

In [15]:
houston_df.head()

,Zip_Code,Neighborhood,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,77002,"Downtown, Midtown",29.75,-95.37,16793,64093.0,208600.0,0.265749,0.416079,0.049119,0.152974,0.071696,0.021696,0.022687
1,77003,East Downtown,29.75,-95.35,10508,36412.0,162600.0,0.318136,0.369392,0.038623,0.165674,0.061130,0.029331,0.017715
2,77004,"South Central, Greater Third Ward",29.72,-95.38,32692,41983.0,191700.0,0.157673,0.345377,0.040444,0.230725,0.137109,0.060271,0.028400
3,77005,West University Place,29.72,-95.42,25528,159732.0,754000.0,0.016033,0.122208,0.040450,0.334924,0.227403,0.184077,0.074904
4,77006,"Montrose, Hyde Park",29.74,-95.39,19664,67458.0,327300.0,0.033678,0.264526,0.030957,0.339017,0.172018,0.101215,0.058589


In [16]:
houston_df.tail()

,Zip_Code,Neighborhood,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
229,77665,Winnie,29.80,-94.30,6446,50149.0,116400.0,0.249186,0.575733,0.035831,0.100434,0.030130,0.005157,0.003529
230,78931,Bleiblerville,30.03,-96.44,323,67917.0,116400.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
231,78933,Cat Spring,29.80,-96.40,1433,59974.0,225700.0,0.049579,0.610851,0.037418,0.211413,0.061740,0.020580,0.008419
232,78944,Industry,30.00,-96.50,562,47083.0,154900.0,0.230461,0.561122,0.068136,0.036072,0.102204,0.002004,0.000000
233,78950,New Ulm,29.90,-96.50,2038,49261.0,170900.0,0.086690,0.559408,0.134115,0.150943,0.056604,0.012239,0.000000


In [17]:
houston_venues = get_nearby_venues(names = houston_df['Neighborhood'],
                                   latitudes = houston_df['lat'],
                                   longitudes = houston_df['lng']
                                  )

In [18]:
houston_venues.shape

(19774, 7)

In [19]:
houston_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Downtown, Midtown",29.75,-95.37,"Spec's Wines, Spirits & Finer Foods",29.748694,-95.376949,Liquor Store
1,"Downtown, Midtown",29.75,-95.37,House of Blues,29.753836,-95.363932,Music Venue
2,"Downtown, Midtown",29.75,-95.37,13 Celsius,29.741267,-95.373784,Wine Bar
3,"Downtown, Midtown",29.75,-95.37,Discovery Green,29.753238,-95.359483,Park
4,"Downtown, Midtown",29.75,-95.37,Weights + Measures,29.742780,-95.372707,Cocktail Bar


In [20]:
houston_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
19769,New Ulm,29.9,-96.5,Lindemann Groceries/Meat Market,29.971723,-96.502287,Grocery Store
19770,New Ulm,29.9,-96.5,The Falls Resort,29.848073,-96.502562,Golf Course
19771,New Ulm,29.9,-96.5,Miss Bobis Roadhouse,29.975303,-96.404399,Restaurant
19772,New Ulm,29.9,-96.5,BlissWood Bed and Breakfast Ranch,29.836110,-96.362002,Bed & Breakfast
19773,New Ulm,29.9,-96.5,Kramer's Catering,30.020897,-96.591722,Food


In [21]:
houston_venues.to_csv('./Data/houston_venues.csv', index = False)

In [72]:
dallas_venues = pd.read_csv('./Data/dallas_venues.csv')

In [73]:
dallas_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addison,32.96,-96.84,Taste of Chicago,32.950202,-96.839735,Pizza Place
1,Addison,32.96,-96.84,Addison Athletic Club,32.949904,-96.849136,Gym
2,Addison,32.96,-96.84,Addison Circle Park,32.960917,-96.826488,Park
3,Addison,32.96,-96.84,Big D Party Rentals,32.977245,-96.838521,Shop & Service
4,Addison,32.96,-96.84,Best Thai Signature,32.954790,-96.841320,Thai Restaurant


In [74]:
ten_or_less_unique_venues = dallas_venues[['Neighborhood','Venue Category']].groupby('Neighborhood').nunique() < 10

In [75]:
ten_or_less_unique_venues = ten_or_less_unique_venues[ten_or_less_unique_venues['Venue Category'] == True]

In [76]:
ten_or_less_unique_venues

,Neighborhood,Venue Category
Neighborhood,,
Alvord,True,True
Celeste,True,True
Northwest Weatherford,True,True
Poolville,True,True
Whitt,True,True


In [77]:
neighborhoods_to_drop = list(ten_or_less_unique_venues.index.values)

In [78]:
dallas_venues = dallas_venues[~dallas_venues['Neighborhood'].isin(neighborhoods_to_drop)]

In [79]:
dallas_df = dallas_df[~dallas_df['Neighborhood'].isin(neighborhoods_to_drop)] 

In [80]:
print('There are {} uniques categories.'.format(len(dallas_venues['Venue Category'].unique())))

There are 279 uniques categories.


In [81]:
dallas_onehot = pd.get_dummies(dallas_venues[['Venue Category']], prefix="", prefix_sep="")

dallas_onehot['Neighborhood'] = dallas_venues['Neighborhood'] 

fixed_columns = [dallas_onehot.columns[-1]] + list(dallas_onehot.columns[:-1])
dallas_onehot = dallas_onehot[fixed_columns]

dallas_onehot.head()


,Zoo,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Engineering Building,College Library,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drive-in Theater,Electronics Store,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,RV Park,Racetrack,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,State / Provincial Park,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Travel & Transport,Truck Stop,University,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [82]:
dallas_onehot.shape

(23372, 279)

In [83]:
dallas_grouped = dallas_onehot.groupby('Neighborhood').mean().reset_index()
dallas_grouped.head()

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Engineering Building,College Library,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drive-in Theater,Electronics Store,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,RV Park,Racetrack,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,State / Provincial Park,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Travel & Transport,Truck Stop,University,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,"(Almeta, Bonite, Bella Vista) - Garland",0.00,0.0,0.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

dallas_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
dallas_neighborhoods_venues_sorted['Neighborhood'] = dallas_grouped['Neighborhood']

for ind in np.arange(dallas_grouped.shape[0]):
    dallas_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dallas_grouped.iloc[ind, :], num_top_venues)

dallas_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"(Almeta, Bonite, Bella Vista) - Garland",Burger Joint,Grocery Store,Brewery,Ice Cream Shop,Fast Food Restaurant,Taco Place,Coffee Shop,Steakhouse,Bakery,Movie Theater
1,"(Arlington Heights, Monticello, West Beyer) - ...",Mexican Restaurant,Grocery Store,Sushi Restaurant,Brewery,Seafood Restaurant,Beer Bar,Burger Joint,Pizza Place,Coffee Shop,Liquor Store
2,"(Bluffview, Greenway Parks) - Dallas",Grocery Store,Mexican Restaurant,Burger Joint,Coffee Shop,American Restaurant,Park,New American Restaurant,Brewery,Shopping Mall,Gourmet Shop
3,"(Casa Linda, Lockwood) - Dallas",Grocery Store,Brewery,Burger Joint,Park,Gym / Fitness Center,Trail,Coffee Shop,Steakhouse,Bar,New American Restaurant
4,"(Casa View, Eastwood Hills) - Dallas",Brewery,Grocery Store,Taco Place,Burger Joint,BBQ Joint,Bar,Thai Restaurant,Ice Cream Shop,Fast Food Restaurant,Coffee Shop


In [96]:
dallas_combined_df = pd.concat([dallas_df.set_index('Neighborhood'),dallas_neighborhoods_venues_sorted.set_index('Neighborhood')], axis = 1, sort = False)

In [123]:
dallas_combined_df.reset_index().to_csv('./Data/dallas_combined_df_with_venue_labels.csv', index = False)

In [106]:
dallas_combine_no_labels = pd.concat([dallas_df.set_index('Neighborhood'), dallas_grouped.set_index('Neighborhood')], axis = 1, sort = False)

In [122]:
dallas_combine_no_labels.reset_index().to_csv('./Data/dallas_combined_df.csv', index = False)

In [40]:
houston_venues = pd.read_csv('./Data/houston_venues.csv')

In [70]:
houston_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Downtown, Midtown",29.75,-95.37,"Spec's Wines, Spirits & Finer Foods",29.748694,-95.376949,Liquor Store
1,"Downtown, Midtown",29.75,-95.37,House of Blues,29.753836,-95.363932,Music Venue
2,"Downtown, Midtown",29.75,-95.37,13 Celsius,29.741267,-95.373784,Wine Bar
3,"Downtown, Midtown",29.75,-95.37,Discovery Green,29.753238,-95.359483,Park
4,"Downtown, Midtown",29.75,-95.37,Weights + Measures,29.742780,-95.372707,Cocktail Bar


In [42]:
ten_or_less_unique_venues = houston_venues[['Neighborhood','Venue Category']].groupby('Neighborhood').nunique() < 10

In [43]:
ten_or_less_unique_venues = ten_or_less_unique_venues[ten_or_less_unique_venues['Venue Category'] == True]

In [44]:
ten_or_less_unique_venues

,Neighborhood,Venue Category
Neighborhood,,
Anahuac,True,True
Central Bolivar Peninsula,True,True
Damon,True,True
Devers,True,True
East Bolivar Peninsula,True,True
East Cleveland,True,True
Industry,True,True
New Ulm,True,True
Romayor,True,True


In [45]:
neighborhoods_to_drop = list(ten_or_less_unique_venues.index.values)

In [46]:
houston_venues = houston_venues[~houston_venues['Neighborhood'].isin(neighborhoods_to_drop)]

In [47]:
houston_df = houston_df[~houston_df['Neighborhood'].isin(neighborhoods_to_drop)] 

In [48]:
print('There are {} uniques categories.'.format(len(houston_venues['Venue Category'].unique())))

There are 290 uniques categories.


In [49]:
houston_onehot = pd.get_dummies(houston_venues[['Venue Category']], prefix="", prefix_sep="")

houston_onehot['Neighborhood'] = houston_venues['Neighborhood'] 

fixed_columns = [houston_onehot.columns[-1]] + list(houston_onehot.columns[:-1])
houston_onehot = houston_onehot[fixed_columns]

houston_onehot.head()

,Zoo Exhibit,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Eye Doctor,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Service,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hot Dog Joint,Hotel,Hunan Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,RV Park,Racetrack,Ramen Restaurant,Rental Car Location,Resort,Rest Area,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Student Center,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Truck Stop,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wareho

In [50]:
houston_onehot.shape

(19711, 290)

In [51]:
houston_grouped = houston_onehot.groupby('Neighborhood').mean().reset_index()
houston_grouped.head()

,Neighborhood,Zoo Exhibit,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Eye Doctor,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Service,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hot Dog Joint,Hotel,Hunan Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,RV Park,Racetrack,Ramen Restaurant,Rental Car Location,Resort,Rest Area,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Student Center,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Truck Stop,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wareho

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

houston_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
houston_neighborhoods_venues_sorted['Neighborhood'] = houston_grouped['Neighborhood']

for ind in np.arange(houston_grouped.shape[0]):
    houston_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(houston_grouped.iloc[ind, :], num_top_venues)

houston_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North Humble,Fast Food Restaurant,Mexican Restaurant,Coffee Shop,Grocery Store,Italian Restaurant,Burger Joint,Gym,Golf Course,Department Store,Hardware Store
1,"Addicks, Park Ten",Fast Food Restaurant,Burger Joint,Liquor Store,Steakhouse,Park,Mexican Restaurant,Grocery Store,BBQ Joint,Salad Place,Seafood Restaurant
2,Aldine,Mexican Restaurant,Seafood Restaurant,Coffee Shop,Brewery,Beer Garden,BBQ Joint,American Restaurant,Fast Food Restaurant,Taco Place,Dessert Shop
3,Alief,Cajun / Creole Restaurant,Grocery Store,Liquor Store,Park,Pizza Place,American Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Mexican Restaurant,Bookstore
4,Alvin,Fast Food Restaurant,Pizza Place,Sandwich Place,American Restaurant,Gym / Fitness Center,Seafood Restaurant,Fried Chicken Joint,Discount Store,Mexican Restaurant,Burger Joint


In [108]:
houston_combined_df = pd.concat([houston_df.set_index('Neighborhood'), houston_neighborhoods_venues_sorted.set_index('Neighborhood')], axis = 1, sort = False)

In [109]:
houston_combined_df.head()

,Zip_Code,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
"Downtown, Midtown",77002,29.75,-95.37,16793,64093.0,208600.0,0.265749,0.416079,0.049119,0.152974,0.071696,0.021696,0.022687,Park,Trail,Coffee Shop,Grocery Store,Pizza Place,American Restaurant,Italian Restaurant,Taco Place,Mexican Restaurant,Café
East Downtown,77003,29.75,-95.35,10508,36412.0,162600.0,0.318136,0.369392,0.038623,0.165674,0.061130,0.029331,0.017715,Coffee Shop,Park,Trail,Brewery,Mexican Restaurant,American Restaurant,Hotel,Grocery Store,Pizza Place,BBQ Joint
"South Central, Greater Third Ward",77004,29.72,-95.38,32692,41983.0,191700.0,0.157673,0.345377,0.040444,0.230725,0.137109,0.060271,0.028400,Park,Grocery Store,Trail,New American Restaurant,American Restaurant,Taco Place,Coffee Shop,Sushi Restaurant,Café,Burger Joint
West University Place,77005,29.72,-95.42,25528,159732.0,754000.0,0.016033,0.122208,0.040450,0.334924,0.227403,0.184077,0.074904,Grocery Store,Trail,Park,Ice Cream Shop,Italian Restaurant,Mexican Restaurant,Coffee Shop,New American Restaurant,Burger Joint,Wine Bar
"Montrose, Hyde Park",77006,29.74,-95.39,19664,67458.0,327300.0,0.033678,0.264526,0.030957,0.339017,0.172018,0.101215,0.058589,Park,Trail,Grocery Store,Pizza Place,Coffee Shop,American Restaurant,Sandwich Place,Taco Place,New American Restaurant,Burger Joint


In [121]:
houston_combined_df.reset_index().to_csv('./Data/houston_combined_with_venue_labels_df.csv', index = False)

In [112]:
houston_combine_no_labels = pd.concat([houston_df.set_index('Neighborhood'), houston_grouped.set_index('Neighborhood')], axis = 1, sort = False)

In [113]:
houston_combine_no_labels.head()

,Zip_Code,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree,Zoo Exhibit,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Eye Doctor,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Service,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hot Dog Joint,Hotel,Hunan Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,RV Park,Racetrack,Ramen Restaurant,Rental Car Location,Resort,Rest Area,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Student Center,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Ti

In [120]:
houston_combine_no_labels.reset_index().to_csv('./Data/houston_combined_df.csv', index = False)